# Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

#!pip install investpy
#import investpy as inv
from time import sleep
import time

In [2]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome import service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
def converter_valor(valor):
    if isinstance(valor, str):
        if valor.endswith('B'):
            return float(valor.replace('B', '').replace(' ', '')) * 1_000
        elif valor.endswith('M'):
            return float(valor.replace('M', '').replace(' ', '')) * 1
        elif valor.endswith('K'):
            return float(valor.replace('K', '').replace(' ', '')) * 0.001
        elif valor == 'NA':
            return float('nan')

# Coletando os dados

In [5]:
codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4',
    #'VVBR3', 'BTOW3','ENBR3',GNDI3,IGTA3
    'BRFS3', 'BRKM5',  'CCRO3', 'CIEL3', 'CMIG4', 'COGN3', 'CPFE3',
    'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3',
     'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4',  'GOAU4', 'GOLL4', 'HAPV3',
     'HYPE3', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3',
    'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PRIO3', 'QUAL3',
    'RADL3', 'RAIL3', 'RENT3', 'SANB11', 'SBSP3','SUZB3', 'TAEE11', 'TIMS3', 'TOTS3',
    'UGPA3', 'USIM5', 'VALE3', 'VIVT3', 'WEGE3', 'WIZC3','YDUQ3','SOMA3','BPAC11','MDIA3'
]

# Acertando

In [7]:
refazer_comeco = True

In [40]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
codigos_ibovespa = [
    'ABEV3', 
    #'AZUL4', 
    # 'B3SA3',
    #  'BBAS3', 
    # 'BBDC3', 
    'BBDC4', 
    #'BBSE3', 
    # 'BEEF3', 
    # 'BPAC11', 
    # 'BRAP4',
    ]
if refazer_comeco:
    chrome_driver_path = "/usr/bin/chromedriver"

    service = Service(executable_path=chrome_driver_path)

    options = Options()
    options.add_argument("--headless")  # Executar em modo headless
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-dev-shm-usage")  # Reduz o uso de memória
    options.add_argument("--disable-gpu")  # Desativar o uso de GPU
    options.add_argument("--window-size=1920,1080")  # Definir o tamanho da janela

    navegador = webdriver.Chrome(service=service, options=options)
    #setor_financeiro = ['BBAS3','BBDC3']
    setor_financeiro = {'BBAS3', 'BBDC3','BBDC4','BBSE3','ITUB4','BPAC11','ITUB4','SANB11'}
    lista_dataframes = []
    iteracao=0
    
    for acao in codigos_ibovespa:
        if acao in setor_financeiro: 
            print(f'Execucao da {acao} do setor financeiro.')
            #Comeca o tempo
            start_time = time.time()
            #Indica a pagina web
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            
            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]
            
            if len(datas) == 1:
                string_de_datas = datas[0]  # Obtenha a string que contém todas as datas
                datas = string_de_datas.split('\n')
            
            preco_lucro = []
            preco_vpa  = []
            preco_receita_liquida = []
            preco_fco = []
            preco_fcf = []
            preco_ativo_total = []
            preco_ebit = []
            preco_capital_giro = []
            preco_ncav = []
            ev_ebit = []
            ev_ebitda = []
            ev_receita_liquida=[]
            ev_fco = []
            ev_fcf =[]
            ev_atito_total =[]
            market_cap_empresa = []
            enterprise_value =[]
            dividend_yield = []


            listas_dados = {
            'datas': datas,
            'Preço/Lucro': preco_lucro,
            'Preço/VPA': preco_vpa,
            'Preço/Receita Líquida': preco_receita_liquida,
            'Preço/FCO': preco_fco,
            'Preço/FCF': preco_fcf,
            'Preço/Ativo Total': preco_ativo_total,
            'Preço/EBIT': preco_ebit,
            'Preço/Capital Giro': preco_capital_giro,
            'Preço/NCAV': preco_ncav,
            'EV/EBIT': ev_ebit,
            'EV/EBITDA': ev_ebitda,
            'EV/Receita Líquida': ev_receita_liquida,
            'EV/FCO': ev_fco,
            'EV/FCF': ev_fcf,
            'EV/Ativo Total': ev_atito_total,
            'Market Cap Empresa': market_cap_empresa,
            'Enterprise Value': enterprise_value,
            'Dividend Yield': dividend_yield
        }
            
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[3]/div/div[2]/div/div[2]/main/div[2]/div[1]/section[2]/div/table/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
                
                # Encontrar a tabela com o id
                tabela = navegador.find_element(By.ID, 'precos_relativos')
    
                    # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')
    
                lista_preco_relativo = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        #lista_preco_relativo.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())
                        try:
                            lista_preco_relativo.append(celula.text.replace('R$', '').replace(',', '.').replace('%', '').strip())
                        except:
                            # Reencontrar a célula se o elemento estiver obsoleto
                            celula = navegador.find_element(By.XPATH, celula.get_attribute('xpath'))
                            lista_preco_relativo.append(celula.text.replace('R$', '').replace(',', '.').replace('%', '').strip())

                metricas = [
                    'Preço/Lucro', 'Preço/VPA', 'Preço/Receita Líquida', 'Preço/FCO',
                    'Preço/Ativo Total', 'Market Cap Empresa','Dividend Yield'
                ]
                for i in range(0, len(lista_preco_relativo)):
                    chave = lista_preco_relativo[i]
                    if chave in metricas:
                        valor = lista_preco_relativo[i + 1]
                        listas_dados[chave].append(valor)
                
                preco_fcf.append(np.nan)
                preco_ebit.append(np.nan)
                preco_capital_giro.append(np.nan)
                preco_ncav.append(np.nan)
                ev_ebit.append(np.nan)
                ev_ebitda.append(np.nan)
                ev_receita_liquida.append(np.nan)
                ev_fco.append(np.nan)
                ev_fcf.append(np.nan)
                ev_atito_total.append(np.nan)
                enterprise_value.append(np.nan)
                
            
        else:
            
            print(f'Execucao da {acao} do setor financeiro.')
            #Comeca o tempo
            start_time = time.time()
            #Indica a pagina web
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            
            
            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]
            if len(datas) == 1:
                string_de_datas = datas[0]  # Obtenha a string que contém todas as datas
                datas = string_de_datas.split('\n')
            
            # Encontrar a tabela com o id
            
            
            preco_lucro = []
            preco_vpa  = []
            preco_receita_liquida = []
            preco_fco = []
            preco_fcf = []
            preco_ativo_total = []
            preco_ebit = []
            preco_capital_giro = []
            preco_ncav = []
            ev_ebit = []
            ev_ebitda = []
            ev_receita_liquida=[]
            ev_fco = []
            ev_fcf =[]
            ev_atito_total =[]
            market_cap_empresa = []
            enterprise_value =[]
            dividend_yield = []

            listas_dados = {
            'datas': datas,
            'Preço/Lucro': preco_lucro,
            'Preço/VPA': preco_vpa,
            'Preço/Receita Líquida': preco_receita_liquida,
            'Preço/FCO': preco_fco,
            'Preço/FCF': preco_fcf,
            'Preço/Ativo Total': preco_ativo_total,
            'Preço/EBIT': preco_ebit,
            'Preço/Capital Giro': preco_capital_giro,
            'Preço/NCAV': preco_ncav,
            'EV/EBIT': ev_ebit,
            'EV/EBITDA': ev_ebitda,
            'EV/Receita Líquida': ev_receita_liquida,
            'EV/FCO': ev_fco,
            'EV/FCF': ev_fcf,
            'EV/Ativo Total': ev_atito_total,
            'Market Cap Empresa': market_cap_empresa,
            'Enterprise Value': enterprise_value,
            'Dividend Yield': dividend_yield
        }
            
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[3]/div/div[2]/div/div[2]/main/div[2]/div[1]/section[2]/div/table/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
                 # Encontrar a tabela com o id
                tabela = navegador.find_element(By.ID, 'precos_relativos')
    
                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')
    
                lista_preco_relativo = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_preco_relativo.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())
                metricas = [
                    'Preço/Lucro', 'Preço/VPA', 'Preço/Receita Líquida', 'Preço/FCO', 'Preço/FCF',
                    'Preço/Ativo Total', 'Preço/EBIT', 'Preço/Capital Giro', 'Preço/NCAV',
                    'EV/EBIT', 'EV/EBITDA', 'EV/Receita Líquida', 'EV/FCO', 'EV/FCF', 'EV/Ativo Total',
                    'Market Cap Empresa', 'Enterprise Value', 'Dividend Yield'
                ]
                for i in range(0, len(lista_preco_relativo)):
                    chave = lista_preco_relativo[i]
                    if chave in metricas:
                        valor = lista_preco_relativo[i + 1]
                        listas_dados[chave].append(valor)
            

        df_preco_relativo = pd.DataFrame(listas_dados)

        df_preco_relativo = df_preco_relativo.rename(columns=lambda coluna: coluna.replace('/', '_').replace(' ', '_').lower())
        
        if acao in setor_financeiro:
            for col in df_preco_relativo.columns[(df_preco_relativo.columns == 'market_cap_empresa')]:
                df_preco_relativo[col] = df_preco_relativo[col].apply(converter_valor)   
        else:
            for col in df_preco_relativo.columns[(df_preco_relativo.columns == 'market_cap_empresa') & (df_preco_relativo.columns == 'enterprise_value')]:
                df_preco_relativo[col] = df_preco_relativo[col].apply(converter_valor)
                
        df_preco_relativo['tic'] = acao

        
        
        lista_dataframes.append(df_preco_relativo)
        
        iteracao = iteracao+1
        
        if iteracao % 5 == 0:
            pd.concat(lista_dataframes).to_csv('precos_relativos_trimestrais.csv')
        
        end_time = time.time()
        
        # Calcula o tempo decorrido em segundos
        execution_time_seconds = end_time - start_time
    
        # Calcula o tempo decorrido em minutos
        execution_time_minutes = execution_time_seconds / 60
    
        # Imprime o tempo de execução em minutos
        print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")

Execucao da ABEV3 do setor financeiro.
Tempo de execução da ação ABEV3: 0.96 minutos
Execucao da BBDC4 do setor financeiro.
Tempo de execução da ação BBDC4: 1.53 minutos


In [43]:
pd.concat(lista_dataframes)

,datas,preço_lucro,preço_vpa,preço_receita_líquida,preço_fco,preço_fcf,preço_ativo_total,preço_ebit,preço_capital_giro,preço_ncav,ev_ebit,ev_ebitda,ev_receita_líquida,ev_fco,ev_fcf,ev_ativo_total,market_cap_empresa,enterprise_value,dividend_yield,tic
0,30/09/2024,14.28,2.03,2.42,7.62,9.57,1.35,10.11,28.45,-4.73,9.27,7.04,2.22,6.99,8.78,1.24,199.18 B,182.77 B,5.78,ABEV3
1,30/06/2024,12.85,1.94,2.29,7.13,9.01,1.29,9.61,50.02,-4.16,9.05,6.86,2.16,6.72,8.49,1.22,184.99 B,174.30 B,6.22,ABEV3
2,31/03/2024,13.22,2.21,2.41,7.37,9.44,1.43,10.28,-809.80,-4.04,9.80,7.38,2.30,7.03,9.00,1.37,191.77 B,182.74 B,6.00,ABEV3
3,31/12/2023,13.67,2.51,2.49,8.02,10.51,1.49,10.63,-44.62,-3.41,9.96,7.53,2.33,7.51,9.84,1.40,198.18 B,185.63 B,5.80,ABEV3
4,30/09/2023,13.43,2.24,2.46,8.97,11.97,1.47,10.88,54.65,-4.61,10.14,7.64,2.29,8.37,11.16,1.37,202.59 B,188.90 B,5.93,ABEV3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,31/03/2001,11.04,1.92,1.31,NA,NaN,0.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15700.0,NaN,5.28,BBDC4
94,31/12/2000,9.51,2.04,1.45,NA,NaN,0.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15080.0,NaN,5.15,BBDC4
95,30/09/2000,8.03,1.88,1.27,NA,NaN,0.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11670.0,NaN,7.188.83,BBDC4
96,30/06/2000,11.21,2.57,1.72,NA,NaN,0.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15890.0,NaN,5.315.38,BBDC4


In [42]:
df_preco_relativo

,datas,preço_lucro,preço_vpa,preço_receita_líquida,preço_fco,preço_fcf,preço_ativo_total,preço_ebit,preço_capital_giro,preço_ncav,ev_ebit,ev_ebitda,ev_receita_líquida,ev_fco,ev_fcf,ev_ativo_total,market_cap_empresa,enterprise_value,dividend_yield,tic
0,30/09/2024,10.52,0.92,0.64,-3.73,NaN,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144240.0,NaN,10.50,BBDC4
1,30/06/2024,11.04,0.88,0.60,3.22,NaN,0.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137260.0,NaN,9.50,BBDC4
2,31/03/2024,11.35,0.89,0.60,-42.04,NaN,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139560.0,NaN,8.05,BBDC4
3,31/12/2023,10.42,0.89,0.59,-836.38,NaN,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141480.0,NaN,7.84,BBDC4
4,30/09/2023,12.35,0.98,0.65,3.36,NaN,0.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152910.0,NaN,6.48,BBDC4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,31/03/2001,11.04,1.92,1.31,NA,NaN,0.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15700.0,NaN,5.28,BBDC4
94,31/12/2000,9.51,2.04,1.45,NA,NaN,0.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15080.0,NaN,5.15,BBDC4
95,30/09/2000,8.03,1.88,1.27,NA,NaN,0.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11670.0,NaN,7.188.83,BBDC4
96,30/06/2000,11.21,2.57,1.72,NA,NaN,0.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15890.0,NaN,5.315.38,BBDC4


In [ ]:
df_preco_relativo.applymap(converter_valor)

In [50]:
metricas_dict = {
    'datas': datas,
    'Preço/Lucro': preco_lucro,
    'Preço/VPA': preco_vpa,
    'Preço/Receita Líquida': preco_receita_liquida,
    'Preço/FCO': preco_fco,
    'Preço/FCF': preco_fcf,
    'Preço/Ativo Total': preco_ativo_total,
    'Preço/EBIT': preco_ebit,
    'Preço/Capital Giro': preco_capital_giro,
    'Preço/NCAV': preco_ncav,
    'EV/EBIT': ev_ebit,
    'EV/EBITDA': ev_ebitda,
    'EV/Receita Líquida': ev_receita_liquida,
    'EV/FCO': ev_fco,
    'EV/FCF': ev_fcf,
    'EV/Ativo Total': ev_atito_total,
    'Market Cap Empresa': market_cap_empresa,
    'Enterprise Value': enterprise_value,
    'Dividend Yield': dividend_yield
}
metricas = [
    'Preço/Lucro', 'Preço/VPA', 'Preço/Receita Líquida', 'Preço/FCO', 'Preço/FCF',
    'Preço/Ativo Total', 'Preço/EBIT', 'Preço/Capital Giro', 'Preço/NCAV',
    'EV/EBIT', 'EV/EBITDA', 'EV/Receita Líquida', 'EV/FCO', 'EV/FCF', 'EV/Ativo Total',
    'Market Cap Empresa', 'Enterprise Value', 'Dividend Yield'
]
for i in range(0, len(lista_preco_relativo)):
    chave = lista_preco_relativo[i]
    if chave in metricas_dict:
        valor = lista_preco_relativo[i + 1]
        metricas_dict[chave].append(valor)



In [ ]:
pd.DataFrame(metricas_dict)

In [17]:
data_frame_feito = pd.read_csv('precos_relativos_trimestrais_2.csv').iloc[:,1:]

In [19]:
data_frame_feito.tic.unique()

array(['ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3',
       'BEEF3', 'BPAC11', 'BRAP4', 'BRFS3', 'BRKM5', 'CCRO3', 'CIEL3',
       'CMIG4', 'COGN3', 'CPFE3', 'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3',
       'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3', 'ENGI11',
       'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4', 'GOAU4', 'GOLL4', 'HAPV3',
       'HYPE3', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3',
       'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3',
       'PETR4', 'PRIO3', 'QUAL3', 'RADL3', 'RAIL3', 'RENT3', 'SANB11',
       'SBSP3', 'SUZB3', 'TAEE11', 'TIMS3', 'TOTS3'], dtype=object)

In [20]:
servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
# Iniciar o navegador com o serviço
navegador = webdriver.Chrome(service=servico)
lista_dataframes = []
#setor_financeiro = ['BBAS3','BBDC3']
setor_financeiro = {'BBAS3', 'BBDC3','BBDC4','BBSE3','ITUB4','BPAC11','ITUB4','SANB11','IRBR3'}
iteracao=0
for acao in codigos_ibovespa:
    if acao in data_frame_feito['tic'].unique():
        lista_dataframes.append(data_frame_feito[data_frame_feito.tic == acao])
        print(f'Essa {acao} ja foi')
    else:
        if acao in setor_financeiro: 
            print(f'Execucao da {acao} do setor financeiro.')
            #Comeca o tempo
            start_time = time.time()
            #Indica a pagina web
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            
            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]
            
            if len(datas) == 1:
                string_de_datas = datas[0]  # Obtenha a string que contém todas as datas
                datas = string_de_datas.split('\n')
            
            preco_lucro = []
            preco_vpa  = []
            preco_receita_liquida = []
            preco_fco = []
            preco_fcf = []
            preco_ativo_total = []
            preco_ebit = []
            preco_capital_giro = []
            preco_ncav = []
            ev_ebit = []
            ev_ebitda = []
            ev_receita_liquida=[]
            ev_fco = []
            ev_fcf =[]
            ev_atito_total =[]
            market_cap_empresa = []
            enterprise_value =[]
            dividend_yield = []
            
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[2]/table/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
                
                # Encontrar a tabela com o id
                tabela = navegador.find_element(By.ID, 'pagina_empresa_precos_relativos')
    
                    # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')
    
                lista_preco_relativo = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_preco_relativo.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())
                preco_lucro.append(lista_preco_relativo[1])
                preco_vpa.append(lista_preco_relativo[3])
                preco_receita_liquida.append(lista_preco_relativo[5])
                preco_fco.append(lista_preco_relativo[7])
                preco_fcf.append(np.nan)
                preco_ativo_total.append(lista_preco_relativo[9])
                preco_ebit.append(np.nan)
                preco_capital_giro.append(np.nan)
                preco_ncav.append(np.nan)
                ev_ebit.append(np.nan)
                ev_ebitda.append(np.nan)
                ev_receita_liquida.append(np.nan)
                ev_fco.append(np.nan)
                ev_fcf.append(np.nan)
                ev_atito_total.append(np.nan)
                market_cap_empresa.append(lista_preco_relativo[11])
                enterprise_value.append(np.nan)
                dividend_yield.append(lista_preco_relativo[-1])
            
        else:
            
            print(f'Execucao da {acao}.')
            #Comeca o tempo
            start_time = time.time()
            #Indica a pagina web
            navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
            
            
            # Extrair o texto de todas as datas
            elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
            datas = [elemento.text for elemento in elementos_data]
            if len(datas) == 1:
                string_de_datas = datas[0]  # Obtenha a string que contém todas as datas
                datas = string_de_datas.split('\n')
            
            # Encontrar a tabela com o id
            
            
            preco_lucro = []
            preco_vpa  = []
            preco_receita_liquida = []
            preco_fco = []
            preco_fcf = []
            preco_ativo_total = []
            preco_ebit = []
            preco_capital_giro = []
            preco_ncav = []
            ev_ebit = []
            ev_ebitda = []
            ev_receita_liquida=[]
            ev_fco = []
            ev_fcf =[]
            ev_atito_total =[]
            market_cap_empresa = []
            enterprise_value =[]
            dividend_yield = []
            
            for i in datas:
                elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[2]/table/thead/tr/th[2]/select')
                # Crie um objeto Select
                select = Select(elemento_select)
                # Altere o período selecionando por texto visível
                select.select_by_visible_text(i)
                servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
                 # Encontrar a tabela com o id
                tabela = navegador.find_element(By.ID, 'pagina_empresa_precos_relativos')
    
                # Pegar as linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')
    
                lista_preco_relativo = []
                for linha in linhas:
                    # Pegar as células de cada linha
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_preco_relativo.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())
                preco_lucro.append(lista_preco_relativo[1])
                preco_vpa.append(lista_preco_relativo[3])
                preco_receita_liquida.append(lista_preco_relativo[5])
                preco_fco.append(lista_preco_relativo[7])
                preco_fcf.append(lista_preco_relativo[9])
                preco_ativo_total.append(lista_preco_relativo[11])
                preco_ebit.append(lista_preco_relativo[13])
                preco_capital_giro.append(lista_preco_relativo[15])
                preco_ncav.append(lista_preco_relativo[17])
                ev_ebit.append(lista_preco_relativo[19])
                ev_ebitda.append(lista_preco_relativo[21])
                ev_receita_liquida.append(lista_preco_relativo[23])
                ev_fco.append(lista_preco_relativo[25])
                ev_fcf.append(lista_preco_relativo[27])
                ev_atito_total.append(lista_preco_relativo[29])
                market_cap_empresa.append(lista_preco_relativo[31])
                enterprise_value.append(lista_preco_relativo[33])
                dividend_yield.append(lista_preco_relativo[-1])
            
        listas_dados = {
            'data':datas,
            'preco_lucro': preco_lucro,
            'preco_vpa': preco_vpa,
            'preco_receita_liquida': preco_receita_liquida,
            'preco_fco': preco_fco,
            'preco_fcf': preco_fcf,
            'preco_ativo_total': preco_ativo_total,
            'preco_ebit':preco_ebit,
            'preco_capital_giro': preco_capital_giro,
            'preco_ncav': preco_ncav,
            'ev_ebit': ev_ebit,
            'ev_ebitda': ev_ebitda,
            'ev_receita_liquida': ev_receita_liquida,
            'ev_fco': ev_fco,
            'ev_fcf': ev_fcf,
            'ev_atito_total': ev_atito_total,
            'market_cap_empresa': market_cap_empresa,
            'enterprise_value': enterprise_value,
            'dividend_yield': dividend_yield
        }
        df_preco_relativo = pd.DataFrame(listas_dados)
        
        if acao in setor_financeiro:
            for col in df_preco_relativo.columns[(df_preco_relativo.columns == 'market_cap_empresa')]:
                df_preco_relativo[col] = df_preco_relativo[col].apply(converter_valor)   
        else:
            for col in df_preco_relativo.columns[(df_preco_relativo.columns == 'market_cap_empresa') | (df_preco_relativo.columns == 'enterprise_value')]:
                df_preco_relativo[col] = df_preco_relativo[col].apply(converter_valor)
                
        df_preco_relativo['tic'] = acao
        
        lista_dataframes.append(df_preco_relativo)
        
        iteracao = iteracao+1
        
        if iteracao % 5 == 0:
            pd.concat(lista_dataframes).to_csv('precos_relativos_trimestrais.csv')
        
        end_time = time.time()
        
        # Calcula o tempo decorrido em segundos
        execution_time_seconds = end_time - start_time
    
        # Calcula o tempo decorrido em minutos
        execution_time_minutes = execution_time_seconds / 60
    
        # Imprime o tempo de execução em minutos
        print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")

Essa ABEV3 ja foi
Essa AZUL4 ja foi
Essa B3SA3 ja foi
Essa BBAS3 ja foi
Essa BBDC3 ja foi
Essa BBDC4 ja foi
Essa BBSE3 ja foi
Essa BEEF3 ja foi
Essa BPAC11 ja foi
Essa BRAP4 ja foi
Essa BRFS3 ja foi
Essa BRKM5 ja foi
Essa CCRO3 ja foi
Essa CIEL3 ja foi
Essa CMIG4 ja foi
Essa COGN3 ja foi
Essa CPFE3 ja foi
Essa CRFB3 ja foi
Essa CSAN3 ja foi
Essa CSNA3 ja foi
Essa CVCB3 ja foi
Essa CYRE3 ja foi
Essa ECOR3 ja foi
Essa EGIE3 ja foi
Essa ELET3 ja foi
Essa ELET6 ja foi
Essa EMBR3 ja foi
Essa ENGI11 ja foi
Essa EQTL3 ja foi
Essa EZTC3 ja foi
Essa FLRY3 ja foi
Essa GGBR4 ja foi
Essa GOAU4 ja foi
Essa GOLL4 ja foi
Essa HAPV3 ja foi
Essa HYPE3 ja foi
Essa IRBR3 ja foi
Essa ITSA4 ja foi
Essa ITUB4 ja foi
Essa JBSS3 ja foi
Essa KLBN11 ja foi
Essa LREN3 ja foi
Essa MGLU3 ja foi
Essa MRFG3 ja foi
Essa MRVE3 ja foi
Essa MULT3 ja foi
Essa NTCO3 ja foi
Essa PCAR3 ja foi
Essa PETR3 ja foi
Essa PETR4 ja foi
Essa PRIO3 ja foi
Essa QUAL3 ja foi
Essa RADL3 ja foi
Essa RAIL3 ja foi
Essa RENT3 ja foi
Essa SA

In [21]:
pd.concat(lista_dataframes).to_csv('precos_relativos_trimestrais_3.csv')

In [ ]:
servico = webdriver.chrome.service.Service(ChromeDriverManager().install())
# Iniciar o navegador com o serviço
navegador = webdriver.Chrome(service=servico)
#setor_financeiro = ['BBAS3','BBDC3']
setor_financeiro = {'BBAS3', 'BBDC3','BBDC4','BBSE3','ITUB4','BPAC11'}
lista_dataframes = []
iteracao=0
for acao in codigos_ibovespa:
    print(f'Execucao da {acao} do setor financeiro.')
    #Comeca o tempo
    start_time = time.time()
    #Indica a pagina web
    servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

    # Iniciar o navegador com o serviço
    navegador = webdriver.Chrome(service=servico)
    navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')

    # Extrair o texto de todas as datas //*[@id="tabela_resumo_empresa_precos_relativos"]/thead/tr/th[2]/select
    #elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_dre_3meses"]/thead/tr/th[2]/select/option')
    elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_precos_relativos"]/thead/tr/th[2]/select')
    datas = [elemento.text for elemento in elementos_data]

    if len(datas) == 1:
        string_de_datas = datas[0]  # Obtenha a string que contém todas as datas
        datas = string_de_datas.split('\n')
    preco_lucro = []
    preco_vpa  = []
    preco_receita_liquida = []
    preco_fco = []
    preco_fcf = []
    preco_ativo_total = []
    preco_ebit = []
    preco_capital_giro = []
    preco_ncav = []
    ev_ebit = []
    ev_ebitda = []
    ev_receita_liquida=[]
    ev_fco = []
    ev_fcf =[]
    ev_atito_total =[]
    market_cap_empresa = []
    enterprise_value =[]
    dividend_yield = []

    for i in datas:

        elemento_select = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[1]/div[2]/table/thead/tr/th[2]/select')
        # Crie um objeto Select
        select = Select(elemento_select)
        # Altere o período selecionando por texto visível
        select.select_by_visible_text(i)
        servico = webdriver.chrome.service.Service(ChromeDriverManager().install())

        # Encontrar a tabela com o id
        tabela = navegador.find_element(By.ID, 'pagina_empresa_precos_relativos')

        # Pegar as linhas da tabela
        linhas = tabela.find_elements(By.TAG_NAME, 'tr')

        lista_preco_relativo = []
        for linha in linhas:
            # Pegar as células de cada linha
            celulas = linha.find_elements(By.TAG_NAME, 'td')
            for celula in celulas:
                lista_preco_relativo.append(celula.text.replace('R$','').replace(',','.').replace('%','').strip())

        preco_lucro.append(lista_preco_relativo[1])
        preco_vpa.append(lista_preco_relativo[3])
        preco_receita_liquida.append(lista_preco_relativo[5])
        preco_fco.append(lista_preco_relativo[7])
        preco_fcf.append(lista_preco_relativo[9])
        preco_ativo_total.append(lista_preco_relativo[11])
        preco_ebit.append(lista_preco_relativo[13])
        preco_capital_giro.append(lista_preco_relativo[15])
        preco_ncav.append(lista_preco_relativo[17])
        ev_ebit.append(lista_preco_relativo[19])
        ev_ebitda.append(lista_preco_relativo[21])
        ev_receita_liquida.append(lista_preco_relativo[23])
        ev_fco.append(lista_preco_relativo[25])
        ev_fcf.append(lista_preco_relativo[27])
        ev_atito_total.append(lista_preco_relativo[29])
        market_cap_empresa.append(lista_preco_relativo[31])
        enterprise_value.append(lista_preco_relativo[33])
        dividend_yield.append(lista_preco_relativo[-1])

    listas_dados = {
            'data':datas,
            'preco_lucro': preco_lucro,
            'preco_vpa': preco_vpa,
            'preco_receita_liquida': preco_receita_liquida,
            'preco_fco': preco_fco,
            'preco_fcf': preco_fcf,
            'preco_ativo_total': preco_ativo_total,
            'preco_ebit':preco_ebit,
            'preco_capital_giro': preco_capital_giro,
            'preco_ncav': preco_ncav,
            'ev_ebit': ev_ebit,
            'ev_ebitda': ev_ebitda,
            'ev_receita_liquida': ev_receita_liquida,
            'ev_fco': ev_fco,
            'ev_fcf': ev_fcf,
            'ev_atito_total': ev_atito_total,
            'market_cap_empresa': market_cap_empresa,
            'enterprise_value': enterprise_value,
            'dividend_yield': dividend_yield
        }
    df_preco_relativo = pd.DataFrame(listas_dados)
    
    lista_dataframes.append(df_preco_relativo[1:])
    
    pd.concat(lista_dataframes).to_csv('precos_relativos_trimestrais.csv')
    
    end_time = time.time()
    
    # Calcula o tempo decorrido em segundos
    execution_time_seconds = end_time - start_time

    # Calcula o tempo decorrido em minutos
    execution_time_minutes = execution_time_seconds / 60

    # Imprime o tempo de execução em minutos
    print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")

In [5]:
import sys
import pandas as pd
sys.path.append('..')
from FundamentsStockBrazil.RelativePrices import PrecosRelativosDataScraper
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options


setor_financeiro = {'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'ITUB4', 'BPAC11', 'ITUB4', 'SANB11', 'IRBR3'}

chrome_driver_path = "/usr/bin/chromedriver"

service = Service(executable_path=chrome_driver_path)

options = Options()
options.add_argument("--headless")  # Executar em modo headless
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-dev-shm-usage")  # Reduz o uso de memória
options.add_argument("--disable-gpu")  # Desativar o uso de GPU
options.add_argument("--window-size=1920,1080")  # Definir o tamanho da janela

codigos_ibovespa = [#'BBAS3',
                    #'BBDC3', 
                    'PETR4'
                    # 'BBDC4', 'BBSE3', 'ITUB4', 'BPAC11', 'ITUB4', 'SANB11', 'IRBR3'
                     ]

scraper = PrecosRelativosDataScraper(setor_financeiro, options, service, acoes=codigos_ibovespa, diretorio='../dados/precos_relativos.csv')

In [2]:
dados = scraper.rodar_acoes()

Tempo de execução da ação PETR4: 3.90 minutos


In [6]:
pd.to_numeric(dados['preço_ncav'], errors='coerce')

0    -0.68
1    -0.67
2    -0.68
3    -0.84
4    -0.70
      ... 
94   -1.75
95   -1.61
96   -1.63
97   -1.47
98   -1.15
Name: preço_ncav, Length: 99, dtype: float64

In [6]:



navegador = scraper.navegador_get(acao="BBDC4")
datas = scraper.obter_datas(navegador=navegador)

df_dados = scraper.coletar_dados_financeiros(
                            navegador=navegador, datas=datas
                        )

In [7]:
df_dados

,datas,preço_lucro,preço_vpa,preço_receita_líquida,preço_fco,preço_fcf,preço_ativo_total,preço_ebit,preço_capital_giro,preço_ncav,ev_ebit,ev_ebitda,ev_receita_líquida,ev_fco,ev_fcf,ev_ativo_total,market_cap_empresa,enterprise_value,dividend_yield
0,Atual - 01/01/2025,"8,46","0,74","0,52","-3,00",NaN,"0,06",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"R$ 118,12 B",NaN,"9,64%"
1,30/09/2024,"10,52","0,92","0,64","-3,73",NaN,"0,08",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"R$ 144,24 B",NaN,"10,50%"
2,30/06/2024,"11,04","0,88","0,60","3,22",NaN,"0,07",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"R$ 137,26 B",NaN,"9,50%"
3,31/03/2024,"11,35","0,89","0,60","-42,04",NaN,"0,08",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"R$ 139,56 B",NaN,"8,05%"
4,31/12/2023,"10,42","0,89","0,59","-836,38",NaN,"0,08",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"R$ 141,48 B",NaN,"7,84%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,31/03/2001,"11,04","1,92","1,31",NA,NaN,"0,23",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"R$ 15,70 B",NaN,"5,28%"
95,31/12/2000,"9,51","2,04","1,45",NA,NaN,"0,26",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"R$ 15,08 B",NaN,"5,15%"
96,30/09/2000,"8,03","1,88","1,27",NA,NaN,"0,21",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"R$ 11,67 B",NaN,"7.188,83%"
97,30/06/2000,"11,21","2,57","1,72",NA,NaN,"0,29",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"R$ 15,89 B",NaN,"5.315,38%"


In [10]:
df_dados["dividend_yield"].str.replace('%','').str.replace(',','.')

0         9.64
1        10.50
2         9.50
3         8.05
4         7.84
        ...   
94        5.28
95        5.15
96    7.188.83
97    5.315.38
98    6.644.23
Name: dividend_yield, Length: 99, dtype: object

In [12]:
pd.to_numeric(df_dados["dividend_yield"].str.replace('%', '').str.replace(',', '.'),errors='coerce')

0      9.64
1     10.50
2      9.50
3      8.05
4      7.84
      ...  
94     5.28
95     5.15
96      NaN
97      NaN
98      NaN
Name: dividend_yield, Length: 99, dtype: float64